# Task 1: Exploratory Data Analysis and Data Preprocessing

## Objective
Understand the structure, content, and quality of the CFPB complaint data and prepare it for the RAG pipeline.


## 1. Import Libraries


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from pathlib import Path

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Set up paths - more robust path resolution
# Try to find project root by looking for data directory
current_dir = Path().resolve()
project_root = current_dir

# Check if we're in the project root (has data/raw directory)
if (current_dir / "data" / "raw").exists():
    project_root = current_dir
# Check if we're in notebooks directory (go up one level)
elif (current_dir.parent / "data" / "raw").exists():
    project_root = current_dir.parent
# Otherwise, try going up two levels (if run from notebooks subdirectory)
elif (current_dir.parent.parent / "data" / "raw").exists():
    project_root = current_dir.parent.parent
else:
    # Fallback: assume current directory is project root
    project_root = current_dir

PROJECT_ROOT = project_root
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"

# Create processed directory if it doesn't exist
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

print(f"Current working directory: {current_dir}")
print(f"Project root: {PROJECT_ROOT}")
print(f"Data raw directory: {DATA_RAW}")
print(f"Data raw exists: {DATA_RAW.exists()}")
print(f"Data processed directory: {DATA_PROCESSED}")


Current working directory: C:\Users\USER\Desktop\Week7\Intelligent-Complaint-Analysis-for-Financial-Services\notebooks
Project root: C:\Users\USER\Desktop\Week7\Intelligent-Complaint-Analysis-for-Financial-Services
Data raw directory: C:\Users\USER\Desktop\Week7\Intelligent-Complaint-Analysis-for-Financial-Services\data\raw
Data raw exists: True
Data processed directory: C:\Users\USER\Desktop\Week7\Intelligent-Complaint-Analysis-for-Financial-Services\data\processed


## 2. Load the CFPB Complaint Dataset


In [3]:
# Load the full CFPB complaint dataset
# Note: Update the filename based on the actual dataset file
# Common filenames: complaints.csv, consumer_complaints.csv, etc.

# Initialize df as empty DataFrame first (in case of errors)
df = pd.DataFrame()

try:
    csv_files = list(DATA_RAW.glob("*.csv"))
    if csv_files:
        data_file = csv_files[0]
        file_size_gb = data_file.stat().st_size / (1024**3)
        print(f"Found CSV file: {data_file}")
        print(f"File size: {file_size_gb:.2f} GB")
        
        # For very large files (>2GB), use optimized chunked reading
        if file_size_gb > 2:
            print("Large file detected. Using optimized chunked loading...")
            print("This may take 5-15 minutes depending on your system...")
            print("-" * 60)
            
            # Use larger chunks and process more efficiently
            chunk_size = 100000  # Read 100k rows at a time (larger chunks = fewer concatenations)
            chunks = []
            total_rows = 0
            
            try:
                # First pass: count total rows and load chunks
                chunk_reader = pd.read_csv(data_file, low_memory=False, chunksize=chunk_size)
                
                for i, chunk in enumerate(chunk_reader):
                    chunks.append(chunk)
                    total_rows += len(chunk)
                    
                    # Show progress every 5 chunks
                    if (i + 1) % 5 == 0:
                        print(f"  Processed {i + 1} chunks (~{total_rows:,} rows loaded)", end='\r')
                
                print(f"\n  Loaded {len(chunks)} chunks with {total_rows:,} total rows")
                print("  Combining chunks (this may take a few minutes)...")
                
                # More memory-efficient concatenation: combine in batches
                # Combine chunks in groups to reduce memory spikes
                batch_size = 20  # Combine 20 chunks at a time
                combined_chunks = []
                
                for i in range(0, len(chunks), batch_size):
                    batch = chunks[i:i+batch_size]
                    combined = pd.concat(batch, ignore_index=True)
                    combined_chunks.append(combined)
                    # Free memory
                    del batch
                    if (i // batch_size + 1) % 5 == 0:
                        print(f"  Combined {min(i+batch_size, len(chunks))}/{len(chunks)} chunks...", end='\r')
                
                # Final combination
                print(f"\n  Final combination of {len(combined_chunks)} batches...")
                df = pd.concat(combined_chunks, ignore_index=True)
                
                # Clean up
                del chunks, combined_chunks
                import gc
                gc.collect()
                
            except MemoryError:
                print("\n⚠ Memory error during chunked loading.")
                print("Trying with smaller chunks...")
                # Fallback: smaller chunks
                chunk_list = []
                for i, chunk in enumerate(pd.read_csv(data_file, low_memory=False, chunksize=25000)):
                    chunk_list.append(chunk)
                    if (i + 1) % 20 == 0:
                        print(f"  Loaded {i + 1} chunks...", end='\r')
                print(f"\n  Combining {len(chunk_list)} chunks...")
                df = pd.concat(chunk_list, ignore_index=True)
                del chunk_list
                gc.collect()
        else:
            print("Loading data (this may take a few minutes)...")
            df = pd.read_csv(data_file, low_memory=False)
        
        print(f"\n✓ Data loaded successfully!")
        print(f"✓ Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
        print(f"✓ Memory usage: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
        
    else:
        print("⚠ No CSV file found in data/raw directory.")
        print("Please download the CFPB complaint dataset and place it in data/raw/")
        print("Dataset can be found at: https://www.consumerfinance.gov/data-research/consumer-complaints/")
        print("\nCreated empty DataFrame for demonstration.")
        
except MemoryError as e:
    print(f"\n⚠ Memory error: The file is too large to load into memory.")
    print("Consider using a machine with more RAM or processing the data in smaller batches.")
    df = pd.DataFrame()
except KeyboardInterrupt:
    print("\n\n⚠ Loading interrupted by user.")
    print("Partial data may be available. Re-run this cell to load the full dataset.")
    df = pd.DataFrame()
except Exception as e:
    print(f"\n⚠ Error loading data: {str(e)}")
    print("Created empty DataFrame. Please check the file path and try again.")
    import traceback
    traceback.print_exc()
    df = pd.DataFrame()

# Verify df is defined
if 'df' not in globals():
    df = pd.DataFrame()

if df.empty:
    print("\n⚠ Warning: DataFrame is empty. Please ensure the data file is in data/raw/")
else:
    print(f"\n✓ DataFrame ready with {len(df):,} rows")
    print("✓ You can now proceed to the next cells.")


Found CSV file: C:\Users\USER\Desktop\Week7\Intelligent-Complaint-Analysis-for-Financial-Services\data\raw\complaints.csv
File size: 5.63 GB
Large file detected. Using optimized chunked loading...
This may take 5-15 minutes depending on your system...
------------------------------------------------------------
  Processed 45 chunks (~4,500,000 rows loaded)

⚠ Loading interrupted by user.
Partial data may be available. Re-run this cell to load the full dataset.

⚠ Warning: DataFrame is empty. Please ensure the data file is in data/raw/


## 3. Initial Data Exploration


In [ ]:
# Display basic information about the dataset
# Check if df is defined and not empty
try:
    # Try to access df
    if 'df' not in globals() or not isinstance(df, pd.DataFrame) or df.empty:
        print("⚠ Error: DataFrame 'df' is not loaded or is empty.")
        print("Please run the data loading cell (Cell 4) first and wait for it to complete.")
        print("The data file is large (5.63 GB) and may take several minutes to load.")
        print("\nSteps to fix:")
        print("1. Go to Cell 4 (Load the CFPB Complaint Dataset)")
        print("2. Run Cell 4 and wait for it to finish (look for '✓ DataFrame ready')")
        print("3. Then come back and run this cell")
    else:
        print("Dataset Shape:", df.shape)
        print("\nColumn Names:")
        print(df.columns.tolist())
        print("\nFirst few rows:")
        display(df.head())
except NameError:
    print("⚠ Error: DataFrame 'df' is not defined.")
    print("Please run Cell 4 (Load the CFPB Complaint Dataset) first.")
    print("Make sure Cell 4 completes successfully before running this cell.")
except Exception as e:
    print(f"⚠ Error: {str(e)}")
    print("Please ensure Cell 4 has been run successfully.")


NameError: name 'df' is not defined

In [4]:
# Data types and missing values
# Check if df is defined and not empty
if 'df' not in globals() or df.empty:
    print("⚠ Error: DataFrame 'df' is not loaded or is empty.")
    print("Please run the data loading cell (Cell 4) first and wait for it to complete.")
else:
    print("Data Types:")
    print(df.dtypes)
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nMissing Values Percentage:")
    print((df.isnull().sum() / len(df) * 100).round(2))


⚠ Error: DataFrame 'df' is not loaded or is empty.
Please run the data loading cell (Cell 4) first and wait for it to complete.


## 4. Identify Key Columns

Based on CFPB dataset structure, we need to identify:
- Product category column
- Consumer complaint narrative column
- Other relevant metadata columns


In [ ]:
# Common column names in CFPB dataset
# Adjust these based on actual column names in your dataset

# Try to identify columns automatically
product_col = None
narrative_col = None

# Common variations of column names in CFPB dataset
# Product column is typically named "Product" or similar
product_keywords = ['product']
# Narrative column is typically "Consumer complaint narrative" or similar
narrative_keywords = ['consumer_complaint_narrative', 'complaint_narrative', 'consumer complaint narrative']

# Try exact matches first (most common CFPB column names)
if 'Product' in df.columns:
    product_col = 'Product'
elif 'product' in df.columns:
    product_col = 'product'
    
if 'Consumer complaint narrative' in df.columns:
    narrative_col = 'Consumer complaint narrative'
elif 'consumer_complaint_narrative' in df.columns:
    narrative_col = 'consumer_complaint_narrative'

# If exact matches not found, try fuzzy matching
if not product_col:
    for col in df.columns:
        col_lower = col.lower().replace('_', ' ').replace('-', ' ')
        if any(keyword in col_lower for keyword in product_keywords):
            product_col = col
            break

if not narrative_col:
    for col in df.columns:
        col_lower = col.lower().replace('_', ' ').replace('-', ' ')
        if any(keyword in col_lower for keyword in narrative_keywords):
            narrative_col = col
            break

print(f"Product column: {product_col}")
print(f"Narrative column: {narrative_col}")

# Display unique products if found
if product_col and not df.empty:
    print(f"\nUnique Products ({df[product_col].nunique()}):")
    print(df[product_col].value_counts())
    
    # Show sample of products
    print(f"\nSample product names (first 10):")
    print(df[product_col].unique()[:10])


In [ ]:
## 5. Distribution Analysis: Products

# Analyze distribution of complaints across different products
if product_col and not df.empty:
    print("=" * 60)
    print("PRODUCT DISTRIBUTION ANALYSIS")
    print("=" * 60)
    
    product_counts = df[product_col].value_counts()
    product_percentages = (df[product_col].value_counts(normalize=True) * 100).round(2)
    
    print("\nComplaints by Product:")
    print(product_counts)
    
    print("\nPercentage Distribution:")
    print(product_percentages)
    
    # Visualize product distribution
    plt.figure(figsize=(14, 6))
    product_counts.plot(kind='bar')
    plt.title('Distribution of Complaints by Product', fontsize=16, fontweight='bold')
    plt.xlabel('Product', fontsize=12)
    plt.ylabel('Number of Complaints', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Pie chart for better visualization
    plt.figure(figsize=(10, 8))
    top_products = product_counts.head(10)  # Top 10 products
    plt.pie(top_products.values, labels=top_products.index, autopct='%1.1f%%', startangle=90)
    plt.title('Top 10 Products by Complaint Volume', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print("Cannot analyze product distribution - product column or data not found.")


In [ ]:
## 6. Consumer Complaint Narrative Analysis

# Analyze the Consumer complaint narrative field
if narrative_col and not df.empty:
    print("=" * 60)
    print("NARRATIVE ANALYSIS")
    print("=" * 60)
    
    # Identify complaints with and without narratives
    df['has_narrative'] = df[narrative_col].notna() & (df[narrative_col].str.strip() != '')
    
    print(f"\nTotal complaints: {len(df):,}")
    print(f"Complaints WITH narrative: {df['has_narrative'].sum():,} ({(df['has_narrative'].sum() / len(df) * 100):.2f}%)")
    print(f"Complaints WITHOUT narrative: {(~df['has_narrative']).sum():,} ({(~df['has_narrative'].sum() / len(df) * 100):.2f}%)")
    
    # Calculate word count for narratives
    def count_words(text):
        if pd.isna(text) or text == '':
            return 0
        return len(str(text).split())
    
    df['narrative_word_count'] = df[narrative_col].apply(count_words)
    
    # Statistics on narrative length
    print("\nNarrative Length Statistics (Word Count):")
    print(df[df['has_narrative']]['narrative_word_count'].describe())
    
    # Visualize narrative length distribution
    plt.figure(figsize=(14, 6))
    
    # Subplot 1: Histogram
    plt.subplot(1, 2, 1)
    narratives_with_data = df[df['has_narrative']]['narrative_word_count']
    plt.hist(narratives_with_data, bins=50, edgecolor='black', alpha=0.7)
    plt.title('Distribution of Narrative Word Counts', fontsize=14, fontweight='bold')
    plt.xlabel('Word Count', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.axvline(narratives_with_data.median(), color='r', linestyle='--', 
                label=f'Median: {narratives_with_data.median():.0f} words')
    plt.axvline(narratives_with_data.mean(), color='g', linestyle='--', 
                label=f'Mean: {narratives_with_data.mean():.0f} words')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Subplot 2: Box plot (zoomed in for readability)
    plt.subplot(1, 2, 2)
    # Filter out extreme outliers for better visualization
    q1 = narratives_with_data.quantile(0.25)
    q3 = narratives_with_data.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 3 * iqr  # More lenient than standard 1.5*IQR
    upper_bound = q3 + 3 * iqr
    filtered_narratives = narratives_with_data[(narratives_with_data >= lower_bound) & 
                                                (narratives_with_data <= upper_bound)]
    plt.boxplot(filtered_narratives, vert=True)
    plt.title('Narrative Word Count (Outliers Filtered)', fontsize=14, fontweight='bold')
    plt.ylabel('Word Count', fontsize=12)
    plt.grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Identify very short and very long narratives
    short_threshold = 10  # Less than 10 words
    long_threshold = 1000  # More than 1000 words
    
    short_narratives = df[df['narrative_word_count'] < short_threshold]
    long_narratives = df[df['narrative_word_count'] > long_threshold]
    
    print(f"\nVery short narratives (< {short_threshold} words): {len(short_narratives):,}")
    print(f"Very long narratives (> {long_threshold} words): {len(long_narratives):,}")
    
    if len(short_narratives) > 0:
        print("\nSample of very short narratives:")
        print(short_narratives[[product_col, narrative_col, 'narrative_word_count']].head(3))
    
    if len(long_narratives) > 0:
        print("\nSample of very long narratives:")
        print(long_narratives[[product_col, narrative_col, 'narrative_word_count']].head(3))
else:
    print("Cannot analyze narratives - narrative column or data not found.")


In [ ]:
## 7. Filter Dataset for Project Requirements

# Filter to include only the specified products and remove empty narratives
if not df.empty and product_col and narrative_col:
    print("=" * 60)
    print("FILTERING DATASET")
    print("=" * 60)
    
    print(f"\nInitial dataset size: {len(df):,} complaints")
    
    # Define the target products (case-insensitive patterns)
    # The task mentions these 4 products (handling both singular and plural):
    target_patterns = [
        r'credit\s*card',           # Matches "Credit card", "Credit Card", "Credit cards", etc.
        r'personal\s*loan',         # Matches "Personal loan", "Personal loans", etc.
        r'savings\s*account',       # Matches "Savings account", "Savings Account", "Savings accounts", etc.
        r'money\s*transfer'         # Matches "Money transfer", "Money Transfer", "Money transfers", etc.
    ]
    
    # Normalize product names for matching
    df['product_normalized'] = df[product_col].astype(str).str.strip().str.lower()
    
    # Find products that match our targets (case-insensitive regex matching)
    matching_mask = pd.Series([False] * len(df))
    matching_products = set()
    
    for pattern in target_patterns:
        mask = df['product_normalized'].str.contains(pattern, case=False, na=False, regex=True)
        matching_mask = matching_mask | mask
        if mask.any():
            matched = df[mask][product_col].unique()
            matching_products.update(matched.tolist())
    
    print(f"\nFound product variations matching targets:")
    for product in sorted(matching_products):
        count = df[df[product_col] == product].shape[0]
        print(f"  - '{product}': {count:,} complaints")
    
    # Filter by product
    if matching_products:
        df_filtered = df[matching_mask].copy()
        print(f"\nAfter filtering by product: {len(df_filtered):,} complaints")
        
        # Remove records with empty narratives
        df_filtered = df_filtered[df_filtered['has_narrative']].copy()
        print(f"After removing empty narratives: {len(df_filtered):,} complaints")
        
        # Drop the temporary columns
        df_filtered = df_filtered.drop(columns=['has_narrative', 'product_normalized'], errors='ignore')
        
        # Show final distribution
        print("\nFinal distribution by product:")
        product_dist = df_filtered[product_col].value_counts()
        for product, count in product_dist.items():
            pct = (count / len(df_filtered) * 100)
            print(f"  - {product}: {count:,} ({pct:.2f}%)")
        
        print(f"\n✓ Filtering complete! Final dataset size: {len(df_filtered):,} complaints")
        print(f"✓ Retention rate: {(len(df_filtered)/len(df)*100):.2f}% of original dataset")
    else:
        print("\n⚠ Warning: Could not find matching products in the dataset.")
        print("Available products in dataset:")
        available_products = df[product_col].value_counts().head(15)
        for product, count in available_products.items():
            print(f"  - '{product}': {count:,} complaints")
        print("\n⚠ Proceeding without product filtering. Please verify product names manually.")
        df_filtered = df[df['has_narrative']].copy() if 'has_narrative' in df.columns else df.copy()
else:
    print("Cannot filter dataset - required columns or data not found.")
    print(f"Product column: {product_col}")
    print(f"Narrative column: {narrative_col}")
    df_filtered = df.copy() if not df.empty else pd.DataFrame()


In [ ]:
## 8. Text Cleaning and Normalization

# Clean the text narratives to improve embedding quality
def clean_narrative(text):
    """
    Clean and normalize consumer complaint narratives.
    
    Steps:
    1. Convert to lowercase
    2. Remove common boilerplate text
    3. Remove excessive whitespace
    4. Remove special characters (optional - keeping basic punctuation)
    5. Normalize whitespace
    """
    if pd.isna(text) or text == '':
        return ''
    
    text = str(text)
    
    # Step 1: Convert to lowercase
    text = text.lower()
    
    # Step 2: Remove common boilerplate patterns
    boilerplate_patterns = [
        r'i am writing to file a complaint',
        r'i am writing to complain',
        r'this is a complaint',
        r'complaint number',
        r'case number',
        r'reference number',
        r'dear sir/madam',
        r'dear customer service',
        r'to whom it may concern',
        r'please find attached',
    ]
    
    for pattern in boilerplate_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)
    
    # Step 3: Remove excessive newlines and whitespace
    text = re.sub(r'\n+', ' ', text)  # Replace newlines with space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    
    # Step 4: Remove special characters but keep basic punctuation
    # Keep: letters, numbers, basic punctuation (. , ! ? : ; - ' " ( ) [ ])
    # Remove: other special characters
    text = re.sub(r'[^\w\s.,!?:;\-\(\)\[\]\'"\/]', '', text)
    
    # Step 5: Strip leading/trailing whitespace
    text = text.strip()
    
    return text

# Apply cleaning to the filtered dataset
if not df_filtered.empty and narrative_col:
    print("=" * 60)
    print("TEXT CLEANING")
    print("=" * 60)
    
    print(f"\nCleaning {len(df_filtered):,} narratives...")
    
    # Show sample before cleaning
    print("\nSample narratives BEFORE cleaning:")
    sample_before = df_filtered[narrative_col].iloc[:3].tolist()
    for i, text in enumerate(sample_before, 1):
        print(f"\n{i}. {text[:200]}..." if len(str(text)) > 200 else f"\n{i}. {text}")
    
    # Apply cleaning
    df_filtered['narrative_cleaned'] = df_filtered[narrative_col].apply(clean_narrative)
    
    # Show sample after cleaning
    print("\n\nSample narratives AFTER cleaning:")
    sample_after = df_filtered['narrative_cleaned'].iloc[:3].tolist()
    for i, text in enumerate(sample_after, 1):
        print(f"\n{i}. {text[:200]}..." if len(str(text)) > 200 else f"\n{i}. {text}")
    
    # Calculate word count for cleaned narratives
    df_filtered['narrative_cleaned_word_count'] = df_filtered['narrative_cleaned'].apply(count_words)
    
    # Compare word counts
    print("\n\nWord Count Comparison:")
    print(f"Original average: {df_filtered['narrative_word_count'].mean():.2f} words")
    print(f"Cleaned average: {df_filtered['narrative_cleaned_word_count'].mean():.2f} words")
    print(f"Average reduction: {(df_filtered['narrative_word_count'].mean() - df_filtered['narrative_cleaned_word_count'].mean()):.2f} words")
    
    print("\n✓ Text cleaning complete!")
else:
    print("Cannot clean text - filtered dataset is empty or narrative column not found.")


In [ ]:
## 9. Final Dataset Preparation

# Prepare the final cleaned dataset for saving
if not df_filtered.empty:
    print("=" * 60)
    print("FINAL DATASET PREPARATION")
    print("=" * 60)
    
    # Keep the original narrative column for reference, but use cleaned version as primary
    # For the RAG pipeline, we'll use the cleaned version
    
    # Create a final dataset with all necessary columns
    # Keep all original columns plus cleaned narrative
    final_df = df_filtered.copy()
    
    # Optionally, replace the original narrative with cleaned version
    # Or keep both - we'll keep both for now
    
    print(f"\nFinal dataset shape: {final_df.shape}")
    print(f"Columns: {final_df.columns.tolist()}")
    
    # Remove any rows where cleaned narrative is empty (shouldn't happen, but safety check)
    final_df = final_df[final_df['narrative_cleaned'].str.strip() != ''].copy()
    print(f"After removing empty cleaned narratives: {len(final_df):,} complaints")
    
    # Summary statistics
    print("\nFinal Dataset Summary:")
    print(f"Total complaints: {len(final_df):,}")
    print(f"\nProduct distribution:")
    print(final_df[product_col].value_counts())
    
    print("\nNarrative length statistics (cleaned):")
    print(final_df['narrative_cleaned_word_count'].describe())
else:
    print("Cannot prepare final dataset - filtered dataset is empty.")
    final_df = pd.DataFrame()


In [ ]:
## 10. Save Filtered and Cleaned Dataset

# Save the processed dataset to data/filtered_complaints.csv
if not final_df.empty:
    print("=" * 60)
    print("SAVING DATASET")
    print("=" * 60)
    
    output_file = PROJECT_ROOT / "data" / "filtered_complaints.csv"
    
    # Save to CSV
    final_df.to_csv(output_file, index=False)
    print(f"\n✓ Dataset saved to: {output_file}")
    print(f"✓ Total records saved: {len(final_df):,}")
    
    # Verify the saved file
    if output_file.exists():
        file_size = output_file.stat().st_size / (1024 * 1024)  # Size in MB
        print(f"✓ File size: {file_size:.2f} MB")
    
    # Also save a summary
    summary_file = PROJECT_ROOT / "data" / "processed" / "task1_summary.txt"
    with open(summary_file, 'w') as f:
        f.write("Task 1: EDA and Preprocessing Summary\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Original dataset size: {len(df):,} complaints\n")
        f.write(f"Final filtered dataset size: {len(final_df):,} complaints\n\n")
        f.write(f"Product distribution:\n{final_df[product_col].value_counts().to_string()}\n\n")
        f.write(f"Narrative statistics:\n{final_df['narrative_cleaned_word_count'].describe().to_string()}\n")
    print(f"✓ Summary saved to: {summary_file}")
else:
    print("⚠ Cannot save dataset - final dataset is empty.")


## 11. Key Findings Summary

### EDA Findings:

1. **Dataset Overview**: The CFPB complaint dataset contains [X] complaints across multiple financial products.

2. **Product Distribution**: 
   - The four target products (Credit Card, Personal Loan, Savings Account, Money Transfers) represent [X]% of total complaints
   - [Product] has the highest number of complaints with [X] complaints
   - [Product] has the lowest number with [X] complaints

3. **Narrative Analysis**:
   - Approximately [X]% of complaints have consumer narratives
   - Average narrative length is [X] words (median: [X] words)
   - Very short narratives (<10 words): [X] complaints
   - Very long narratives (>1000 words): [X] complaints

4. **Data Quality**:
   - [X] complaints were removed due to missing narratives
   - [X] complaints were removed as they didn't match target products
   - Final cleaned dataset: [X] complaints

5. **Text Cleaning Impact**:
   - Average word count reduction: [X] words per narrative
   - Boilerplate text patterns identified and removed
   - Text normalized for better embedding quality

### Preprocessing Steps Completed:

✅ Filtered to target products (Credit Card, Personal Loan, Savings Account, Money Transfers)  
✅ Removed complaints with empty narratives  
✅ Applied text cleaning (lowercasing, boilerplate removal, whitespace normalization)  
✅ Saved cleaned dataset to `data/filtered_complaints.csv`  

The cleaned dataset is now ready for Task 2: Text Chunking, Embedding, and Vector Store Indexing.


In [ ]:
# Display final statistics for the report
if not final_df.empty:
    print("\n" + "=" * 60)
    print("FINAL STATISTICS FOR REPORT")
    print("=" * 60)
    
    print(f"\n1. Total complaints in original dataset: {len(df):,}")
    print(f"2. Total complaints after filtering: {len(final_df):,}")
    print(f"3. Percentage retained: {(len(final_df)/len(df)*100):.2f}%")
    
    print(f"\n4. Product Distribution:")
    for product, count in final_df[product_col].value_counts().items():
        pct = (count / len(final_df) * 100)
        print(f"   - {product}: {count:,} ({pct:.2f}%)")
    
    print(f"\n5. Narrative Statistics:")
    print(f"   - Mean word count: {final_df['narrative_cleaned_word_count'].mean():.2f} words")
    print(f"   - Median word count: {final_df['narrative_cleaned_word_count'].median():.2f} words")
    print(f"   - Min word count: {final_df['narrative_cleaned_word_count'].min()} words")
    print(f"   - Max word count: {final_df['narrative_cleaned_word_count'].max()} words")
    
    print("\n✓ Task 1 complete! Ready for Task 2.")
